In [3]:
import pandas as pd
import numpy as np


In [4]:
card_name_df = pd.read_csv('/home/oem/Documents/Extras_Repos/My_Small_Repos/mtg_draft_bot/metadata/STX_card_names.csv')
# draft_csv_path = '/media/oem/8931ed1e-dc55-44ac-b428-d43ad01d9eb3/Data/mtg_draft_bot/17lands/draft_data_public.STX_comma.TradDraft.csv'
draft_csv_path = '/media/oem/8931ed1e-dc55-44ac-b428-d43ad01d9eb3/Data/mtg_draft_bot/17lands/draft_data_public.STX.PremierDraft.csv'
init_df = pd.read_csv(draft_csv_path,nrows=1)
col_names = init_df.columns
pack_cols = [ name
                for name in col_names
                if name.startswith('pack_card_')]

In [5]:
def parse_row(row):
    card_bin = row[pack_cols].values.astype(np.uint8)
    in_pack = np.where(card_bin==1)[0]
    out = {'stx_common':0,'stx_uncommon':0,'stx_rare':0,'stx_mythic':0,'sta_uncommon':0,'sta_rare':0,'sta_mythic':0,'lesson_common':0,'lesson_uncommon':0,'lesson_rare':0,'lesson_mythic':0}
    for in_pack_idx in in_pack:
        set = card_name_df.iloc[in_pack_idx]['Set Code']
        rarity = card_name_df.iloc[in_pack_idx]['Rarity']
        is_lesson = card_name_df.iloc[in_pack_idx]['Type'].endswith('Lesson')
        if is_lesson:
            out['lesson_'+rarity]+=1
        else:
            out[set+'_'+rarity]+=1
    return out


In [6]:
test_df = pd.read_csv(draft_csv_path,nrows=1000)
sub_df = test_df[test_df['pick_number']==0]

In [7]:
aa = sub_df.iloc[0]
parse_row(aa)


{'stx_common': 9,
 'stx_uncommon': 3,
 'stx_rare': 1,
 'stx_mythic': 0,
 'sta_uncommon': 1,
 'sta_rare': 0,
 'sta_mythic': 0,
 'lesson_common': 1,
 'lesson_uncommon': 0,
 'lesson_rare': 0,
 'lesson_mythic': 0}

In [8]:
all_outs = []
for chunk in  pd.read_csv(draft_csv_path,chunksize=10000):
    sub_df = chunk[chunk['pick_number']==0]
    for row_idx,row in sub_df.iterrows():
        out = parse_row(row)
        all_outs.append(out)
print('done parsing')

done parsing


In [9]:
multi_sta = 0
for out in all_outs:
    if out['sta_uncommon'] +out['sta_rare']+out['sta_mythic'] >1:
        multi_sta+=1
print(multi_sta)

0


In [10]:
multi_sta = 0
for out in all_outs:
    if out['sta_uncommon'] +out['sta_rare']+out['sta_mythic'] >1:
        multi_sta+=1
print(multi_sta)

0


In [11]:
#probability that 1 stx_uncommon will be swapped for 1 lesson_uncommon, happens only once per pack, and happens independently of the card in the  (common/rare) lesson slot
total_lesson_uncommon = 0
for out in all_outs:
    if out['lesson_uncommon'] ==1:
        total_lesson_uncommon+=1
    else:
        if out['stx_uncommon'] != 3:
            print("This shouldn't happen")
print(" prob switch : {}".format(total_lesson_uncommon/len(all_outs)))

 prob switch : 0.1865764233993527


In [12]:
#probabilities for the lesson slot(can only be common/uncommon/mythic)
total_lesson_common = 0
total_lesson_rare = 0
total_lesson_mythic = 0
for out in all_outs:
    if out['lesson_common'] ==1:
        total_lesson_common+=1
    elif out['lesson_rare'] ==1:
        total_lesson_rare+=1
    elif out['lesson_mythic'] ==1:
        total_lesson_mythic+=1
    else:
        print('this shouldnt happen')
if total_lesson_common+total_lesson_rare+total_lesson_mythic!=len(all_outs):
    print('this shouldnt happen')
print(" common lesson rate : {}, rare lesson rate : {}, mythic lesson rate : {}".format(total_lesson_common/len(all_outs),
                                                                                         total_lesson_rare/len(all_outs),
                                                                                         total_lesson_mythic/len(all_outs),   ))

 common lesson rate : 0.924774665362472, rare lesson rate : 0.06804351108456039, mythic lesson rate : 0.00718182355296759


In [13]:
total_sta_uncommon = 0
total_sta_rare = 0
total_sta_mythic = 0
for out in all_outs:
    if out['sta_uncommon'] ==1:
        total_sta_uncommon+=1
    elif out['sta_rare'] ==1:
        total_sta_rare+=1
    elif out['sta_mythic'] ==1:
        total_sta_mythic+=1
    else:
        print('this shouldnt happen')
print(" sta uncommon rate : {}, rare sta rate : {}, mythic sta rate : {}".format(total_sta_uncommon/len(all_outs),
                                                                                         total_sta_rare/len(all_outs),
                                                                                         total_sta_mythic/len(all_outs),   ))

 sta uncommon rate : 0.6666819843890527, rare sta rate : 0.26613448522605676, mythic sta rate : 0.0671835303848906


In [15]:
total_stx_rare = 0
total_stx_mythic = 0
for out in all_outs:

    if out['stx_rare'] ==1:
        total_stx_rare+=1
    elif out['stx_mythic'] ==1:
        total_stx_mythic+=1
    else:
        print('this shouldnt happen')
print(" rare stx rate : {}, mythic stx rate : {}".format(
                                                                                         total_stx_rare/len(all_outs),
                                                                                         total_stx_mythic/len(all_outs),   ))

 rare stx rate : 0.8635650467081121, mythic stx rate : 0.13643495329188796
